##### 空间定点宽带Beamforming展示

demo提供了空间定点beamforming语音增强的实现， 采用的是CBF方法， 稍微更改可以变成MVDR方法， 但个人理解MVDR方法只是分辨率稍高，但需要计算矩阵的逆大大增加计算量，因此建议使用传统的CBF方法就好。

本次demo中， 我引入了三个声源， 控制第一个声源音量为-20dB， 另外两个声源的音量为-26dB， 通过麦克风阵列向第一个声源位置进行beamforming波束形成，实现信号增强。



In [1]:
from Beamform.BFModel import beamforming_model
from Beamform.CMA100 import CMA100
import librosa
import numpy as np
import matplotlib.pyplot as plt
from Beamform.tools import ConvertToTargetDb, NormLength, NormAmplitude, CalRms
import IPython.display as ipd
import soundfile

In [2]:
audio_data, _ = librosa.load('./test_audios/1.wav', sr=16000)
noise_data1, _ = librosa.load('./test_audios/A8_0.wav', sr=16000)
noise_data2, _ = librosa.load('./noise/noise1.wav', sr=16000)
audio_data = NormLength(audio_data, num_samples=4*16000)
noise_data1 = NormLength(noise_data1, num_samples=4*16000)
noise_data2 = NormLength(noise_data2, num_samples=4*16000)

audio_data = ConvertToTargetDb(audio_data, -20)
noise_data1 = ConvertToTargetDb(noise_data1, -26)
noise_data2 = ConvertToTargetDb(noise_data2, -26)

# 进行短时傅里叶变换， 准备模拟流式处理
audio_rfft = librosa.stft(audio_data, n_fft=512, win_length=512, hop_length=128)
noise1_rfft = librosa.stft(noise_data1, n_fft=512, win_length=512, hop_length=128)
noise2_rfft = librosa.stft(noise_data2, n_fft=512, win_length=512, hop_length=128)



In [3]:
# 设置我们信号源位置。
audio_x, audio_y, audio_z = 2.0, 0.0, 0.0
noise1_x, noise1_y, noise1_z = -1.4, -1.4, 0.0
noise2_x, noise2_y, noise2_z = -2.0, 0.0, 0.0

# 初始化cma100阵列
cma100 = CMA100(fs= 16000, L=512)

# 模拟阵列流式接受信号， 并处理的过程
frameLength = 512
N = audio_rfft.shape[1]
enhancedAudio = np.zeros_like(audio_rfft)
micAudio = np.zeros_like(audio_rfft)

# 流式处理：
for i in range(N):
    cma100.reset()
    cma100.add_signal(S_x = audio_x, S_y = audio_y, S_z = audio_z, signal_rfft = audio_rfft[:, i]) # 接受第一个信号源
    cma100.add_signal(S_x = noise1_x, S_y = noise1_y, S_z = noise1_z, signal_rfft = noise1_rfft[:, i]) # 接受第二个信号源
    cma100.add_signal(S_x = noise2_x, S_y = noise2_y, S_z = noise2_z, signal_rfft = noise2_rfft[:, i]) # 接受第三个信号源
    
    # 先抽取处理前的信号， 好做对比
    micAudio[:,i] = cma100.build_signal_rfft()

    # beamforming
    cma100.beamforming(audio_x, audio_y, audio_z)
    enhancedAudio[:,i] = cma100.build_signal_rfft()

enhancedAudio = librosa.istft(enhancedAudio, n_fft=512, win_length=512, hop_length=128)
micAudio = librosa.istft(micAudio, n_fft=512, win_length=512, hop_length=128)
enhancedAudio = NormAmplitude(enhancedAudio)
micAudio = NormAmplitude(micAudio)

# 保存数据
soundfile.write("./bf_input.wav", micAudio, samplerate=16000)
soundfile.write("./bf_output.wav", enhancedAudio, samplerate=16000)


In [4]:
# 设置我们信号源位置。
audio_x, audio_y, audio_z = 2.0, 0.0, 0.0
noise1_x, noise1_y, noise1_z = -1.4, -1.4, 0.0
noise2_x, noise2_y, noise2_z = -2.0, 0.0, 0.0

# 初始化cma100阵列
cma100 = CMA100(fs= 16000, L=512)

# 模拟阵列流式接受信号， 并处理的过程
frameLength = 512
N = audio_rfft.shape[1]
enhancedAudio = np.zeros_like(audio_rfft)
micAudio = np.zeros_like(audio_rfft)

# 流式处理：
for i in range(N):
    cma100.reset()
    cma100.add_signal(S_x = audio_x, S_y = audio_y, S_z = audio_z, signal_rfft = audio_rfft[:, i]) # 接受第一个信号源
    cma100.add_signal(S_x = noise1_x, S_y = noise1_y, S_z = noise1_z, signal_rfft = noise1_rfft[:, i]) # 接受第二个信号源
    cma100.add_signal(S_x = noise2_x, S_y = noise2_y, S_z = noise2_z, signal_rfft = noise2_rfft[:, i]) # 接受第三个信号源
    
    # 先抽取处理前的信号， 好做对比
    micAudio[:,i] = cma100.build_signal_rfft()
    cma100.band_filter() # 频点分配

    # beamforming
    cma100.beamforming(audio_x, audio_y, audio_z)
    enhancedAudio[:,i] = cma100.build_signal_rfft()

enhancedAudio = librosa.istft(enhancedAudio, n_fft=512, win_length=512, hop_length=128)
micAudio = librosa.istft(micAudio, n_fft=512, win_length=512, hop_length=128)
enhancedAudio = NormAmplitude(enhancedAudio)
micAudio = NormAmplitude(micAudio)

# 保存数据
soundfile.write("./bf_input.wav", micAudio, samplerate=16000)
soundfile.write("./bf_output.wav", enhancedAudio, samplerate=16000)